# IMDB Sentiment Analysis using LSTM

## Importing libraries

In [1]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split as tts
from keras.layers import Dense, LSTM,Embedding, SpatialDropout1D
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

## Reading the dataset

In [2]:
data=pd.read_csv('IMDB-Dataset.csv')

In [3]:
# columns of the dataset
data.columns

Index(['review', 'sentiment'], dtype='object')

In [4]:
# shape of the data
data.shape

(50000, 2)

In [5]:
# 5 elements from the top
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


## Text Preprocessing

In [6]:
import string
# removing the html tags
def clean_html(text):
    clean=re.compile('<.*?>')
    cleantext=re.sub(clean,'',text)
    return cleantext
    
# first round of cleaning
def clean_text1(text):
    text=text.lower()
    text=re.sub('\[.*?\]','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\w*\d\w*','',text)
    return text

# second round of cleaning
def clean_text2(text):
    text=re.sub('[''"",,,]','',text)
    text=re.sub('\n','',text)
    return text
    
cleaned_html=lambda x:clean_html(x)
cleaned1=lambda x:clean_text1(x)
cleaned2=lambda x:clean_text2(x)

data['review']=pd.DataFrame(data.review.apply(cleaned_html))
data['review']=pd.DataFrame(data.review.apply(cleaned1))
data['review']=pd.DataFrame(data.review.apply(cleaned2))

## Defining the model

In [7]:
max_features=5000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['review'].values)
X = tokenizer.texts_to_sequences(data['review'].values)
X = pad_sequences(X,maxlen=600)

In [8]:
embed_dim = 128
lstm_out = 128

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 600, 128)          640000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 600, 128)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
Total params: 771,842
Trainable params: 771,842
Non-trainable params: 0
_________________________________________________________________
None


## Split the dataset

In [9]:
Y=pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = tts(X,Y, test_size = 0.2, random_state = 42)

## Running the model

In [10]:
batch_size = 64
model.fit(X_train, Y_train, epochs = 8, batch_size=batch_size, verbose = True)

Epoch 1/8
625/625 [==============================] - 340s 544ms/step - loss: 0.4191 - accuracy: 0.8067
Epoch 2/8
625/625 [==============================] - 354s 566ms/step - loss: 0.3087 - accuracy: 0.8748
Epoch 3/8
625/625 [==============================] - 369s 591ms/step - loss: 0.2504 - accuracy: 0.9024
Epoch 4/8
625/625 [==============================] - 316s 506ms/step - loss: 0.2264 - accuracy: 0.9122
Epoch 5/8
625/625 [==============================] - 318s 509ms/step - loss: 0.2091 - accuracy: 0.9202
Epoch 6/8
625/625 [==============================] - 307s 491ms/step - loss: 0.1995 - accuracy: 0.9222
Epoch 7/8
625/625 [==============================] - 307s 491ms/step - loss: 0.2017 - accuracy: 0.9198
Epoch 8/8
625/625 [==============================] - 306s 489ms/step - loss: 0.1783 - accuracy: 0.9313


In [14]:
result=model.evaluate(X_test,Y_test,steps=625)

625/625 [==============================] - 55s 88ms/step - loss: 0.3039 - accuracy: 0.8894
